In [10]:
import openai
import os
from pprint import pprint
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import AzureChatOpenAI

# read local .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

# setup azure openai api
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OAI_ENDPOINT")
# openai.api_version = "2023-03-15-preview"
openai.api_version = "2023-05-15"
openai.api_key  = os.getenv('AZURE_OAI_KEY')

llm_name = "gpt-3.5-turbo"
deployment_name_gpt = os.getenv("AZURE_OAI_MODEL_GPT_3")
deployment_name_ada = os.getenv("AZURE_OAI_MODEL_ADA")

vector_store_address: str = os.getenv("AZURE_SEARCH_ENDPOINT")
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")

os.environ["OPENAI_API_TYPE"] = openai.api_type
os.environ["OPENAI_API_BASE"] = openai.api_base
os.environ["OPENAI_API_KEY"] = openai.api_key
os.environ["OPENAI_API_VERSION"] = openai.api_version

In [6]:
llm = AzureChatOpenAI(deployment_name=deployment_name_gpt)

In [7]:
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=deployment_name_ada, chunk_size=1)
index_name: str = "facts-about-snakes"
azure_search_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [8]:
azure_search_retreiver = azure_search_store.as_retriever(search_kwargs={"k":4})

### chains

In [9]:
from langchain.chains import RetrievalQA
acs_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=azure_search_retreiver)

In [11]:
pprint(acs_qa.run("what are the snakes covered in the document?"))

('The document mentions six types of snakes found in North Carolina. These '
 'include the Copperhead, Timber Rattlesnake, Pigmy Rattlesnake, Eastern '
 'Diamondback Rattlesnake, and Eastern Coral Snake.')


### Tools

In [23]:
from langchain.agents import Tool

description=(
    "Useful FAQ about Snakes in North Carolina. "
    "You should ALWAYS use this tool whenever you need to answer a question about Snakes, "
)

acs_tool = Tool(name="acs_search", description=description, func=acs_qa.run)


In [24]:
pprint(acs_tool)

Tool(name='acs_search', description='Useful FAQ about Snakes in North Carolina. You should ALWAYS use this tool whenever you need to answer a question about Snakes, ', args_schema=None, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, func=<bound method Chain.run of RetrievalQA(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, combine_documents_chain=StuffDocumentsChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, input_key='input_documents', output_key='output_text', llm_chain=LLMChain(memory=None, callbacks=None, callback_manager=None, verbose=False, tags=None, metadata=None, prompt=ChatPromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, templat

### Create Agents from tools

In [25]:
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent

In [26]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
tools = [acs_tool]
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

### Test refined tools

In [37]:
agent_chain.run("What are the venomous snakes in North Carolina?")



> Entering new AgentExecutor chain...
```json
{
    "action": "acs_search",
    "action_input": "Snakes in North Carolina"
}
```
Observation: North Carolina is home to thirty-eight different species of snakes, out of which six are venomous. The venomous snakes include the Copperhead, Cottonmouth, Timber Rattlesnake, Eastern Diamondback Rattlesnake, Pigmy Rattlesnake, and the Eastern Coral Snake. The Copperhead is the most common and widespread venomous snake in North Carolina, occurring in both rural and urban environments.
Thought:```json
{
    "action": "Final Answer",
    "action_input": "The six venomous snake species found in North Carolina are the Copperhead, Cottonmouth, Timber Rattlesnake, Eastern Diamondback Rattlesnake, Pigmy Rattlesnake, and the Eastern Coral Snake. The Copperhead is the most common and widespread venomous snake in North Carolina."
}
```

> Finished chain.


'The six venomous snake species found in North Carolina are the Copperhead, Cottonmouth, Timber Rattlesnake, Eastern Diamondback Rattlesnake, Pigmy Rattlesnake, and the Eastern Coral Snake. The Copperhead is the most common and widespread venomous snake in North Carolina.'

In [38]:
agent_chain.run("give me some facts about Eastern Coral Snake?")



> Entering new AgentExecutor chain...
{
    "action": "acs_search",
    "action_input": "facts about Eastern Coral Snake in North Carolina"
}
Observation: The Eastern Coral Snake is one of the six venomous snakes found in North Carolina. It is also an endangered species. The snake is usually about 20-30 inches long and has distinctive red, yellow, and black banding. It is found in the southeastern United States, including North Carolina, and prefers wooded areas or marshy environments. The Eastern Coral Snake is not aggressive and usually only bites when it feels threatened, so it is important to avoid handling or disturbing it. The venom of the Eastern Coral Snake is highly toxic and can cause serious health problems, including muscle paralysis and respiratory failure.
Thought:{
    "action": "Final Answer",
    "action_input": "The Eastern Coral Snake is an endangered venomous snake species found in North Carolina. It has distinctive red, yellow, and black banding and is usually fo

'The Eastern Coral Snake is an endangered venomous snake species found in North Carolina. It has distinctive red, yellow, and black banding and is usually found in wooded areas or marshy environments. It is not aggressive and usually only bites when it feels threatened. Its venom is highly toxic and can cause serious health problems, including muscle paralysis and respiratory failure.'

In [41]:
memory.load_memory_variables({})

TypeError: 'list' object is not callable